In [1]:
pwd

'C:\\Users\\jonho\\OneDrive - Juniata College\\FA22\\Machine\\Data_Files'

In [2]:
import pandas as pd
import numpy as np

In [16]:
gameData = pd.read_csv('applicationInformation.csv',
                      encoding = 'latin-1')

In [17]:
gameData

,appid,type,name,releasedate,freetoplay
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0
1,570,game,Dota 2,9-Jul-13,1.0
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaN,NaN
4,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0
...,...,...,...,...,...
1995,34000,advertising,Football Manager 2010,29-Oct-09,1.0
1996,202480,NaN,Skyrim Creation Kit,NaN,NaN
1997,429050,game,Feed and Grow: Fish,8-Jan-16,0.0
1998,209650,game,Call of Duty: Advanced Warfare,3-Nov-14,0.0


In [22]:
playerMean = {}

#Group player data by year
def getPlayerCountMean(appid):
    global playerMean
    try:
        playerCountData = pd.read_csv(f'PlayerCountHistoryPart1/{appid}.csv',
                                     index_col=False,
                                     encoding = 'latin-1')
    except FileNotFoundError:
        try:
            playerCountData = pd.read_csv(f'PlayerCountHistoryPart2/{appid}.csv',
                                         index_col=False,
                                         encoding = 'latin-1')
        except FileNotFoundError:
            print(f"ERROR! {appid}.csv not found!")
            playerMean[appid] = (appid, np.NaN, np.NaN, np.NaN, np.NaN)
            return
    
    
    #Convert the Time column to datetime object
    playerCountData = playerCountData.fillna(0)
    playerCountData['Time'] = pd.to_datetime(playerCountData['Time'], errors='coerce')        
    #Group by year
    grouped = playerCountData.groupby(playerCountData.Time.dt.year).mean()
    grouped.columns = ["Mean"]
    grouped["Mean"] = grouped["Mean"].round().astype(int)
    #pivot table to be 1x4
    grouped = grouped.T
    try:
        playerMean[appid] = (appid, grouped[2017][0], grouped[2018][0],grouped[2019][0], grouped[2020][0]) 
    except KeyError:
        print(f"ERROR: appid = {appid}")
        playerMean[appid] = (appid, np.NaN, np.NaN, np.NaN, np.NaN)


In [23]:
gameData['appid'].map(getPlayerCountMean)

0       None
1       None
2       None
3       None
4       None
        ... 
1995    None
1996    None
1997    None
1998    None
1999    None
Name: appid, Length: 2000, dtype: object

In [26]:
gameData['yearlyMeanPlayers'] = gameData['appid'].map(playerMean)
gameData.head()

,appid,type,name,releasedate,freetoplay,yearlyMeanPlayers
0,578080,game,PLAYERUNKNOWN'S BATTLEGROUNDS,21-Dec-17,0.0,"(578080, 1443309, 841204, 366385, 238828)"
1,570,game,Dota 2,9-Jul-13,1.0,"(570, 528727, 450816, 475222, 442868)"
2,730,game,Counter-Strike: Global Offensive,21-Aug-12,1.0,"(730, 347535, 317709, 396300, 660862)"
3,622590,NaN,PLAYERUNKNOWN'S BATTLEGROUNDS (Test Server),NaN,NaN,"(622590, 0, 2811, 580, 587)"
4,359550,game,Tom Clancy's Rainbow Six Siege,1-Dec-15,0.0,"(359550, 60281, 67572, 71262, 90971)"


In [27]:
meanPlayers = pd.DataFrame(gameData['yearlyMeanPlayers'].values.tolist())
meanPlayers.columns = ["appid", "meanPlayers2017", "meanPlayers2018", "meanPlayers2019", "meanPlayers2020"]
meanPlayers.head()

,appid,meanPlayers2017,meanPlayers2018,meanPlayers2019,meanPlayers2020
0,578080,1443309,841204,366385,238828
1,570,528727,450816,475222,442868
2,730,347535,317709,396300,660862
3,622590,0,2811,580,587
4,359550,60281,67572,71262,90971


In [29]:
meanPlayers.to_csv('mean_player_counts.csv')